# Additional Functionality in Other Packages

In [1]:
progress = false

Several packages extend the functionality of MixedModels.jl, both in
ways specific to mixed models and in ways applicable to more general
regression models. In the following, we will use the models from the
previous sections to showcase this functionality.

In [1]:
using MixedModels

In [1]:
insteval = MixedModels.dataset("insteval")
ie1 = fit(MixedModel,
          @formula(y ~ 1 + studage + lectage + service + (1|s) + (1|d) + (1|dept)),
          insteval; progress)


In [1]:
ie2 = fit(MixedModel,
          @formula(y ~ 1 + studage + lectage + service +
                      (1 | s) +
                      (1 + service | d) +
                      (1 + service | dept)),
          insteval; progress)

In [1]:
sleepstudy = MixedModels.dataset("sleepstudy")
ss1 = fit(MixedModel, @formula(reaction ~ 1 + days + (1|subj)), sleepstudy; progress)

In [1]:
ss2 = fit(MixedModel, @formula(reaction ~ 1 + days + (1 + days|subj)), sleepstudy; progress)

In [1]:
using DataFrames
contra = DataFrame(MixedModels.dataset("contra"))
contra[!, :anych] .= contra[!, :livch] .!= "0"
contrasts = Dict(:livch => EffectsCoding(; base="0"),
                 :urban => HelmertCoding(),
                 :anych => HelmertCoding())
gm1 = fit(MixedModel,
          @formula(use ~ 1 + urban + anych * age + abs2(age) + (1 | dist & urban)),
          contra,
          Bernoulli();
          contrasts,
          progress)

# MixedModelsExtras.jl

<https://palday.github.io/MixedModelsExtras.jl/v2>

MixedModelsExtras.jl is a collection of odds-and-ends that may be useful
when working with mixed effects models, but which we do not want to
include in MixedModels.jl at this time. Some functions may one day
migrate to MixedModels.jl, when we are happy with their performance and
interface (e.g. `vif`), but some are intentionally omitted from
MixedModels.jl (e.g. `r2`, `adjr2`).

In [1]:
using MixedModelsExtras

In [1]:
r2(ss2; conditional=true)

0.8263131642760502

In [1]:
r2(ss2; conditional=false)

0.28647139510771

In [1]:
icc(ie2)

0.28853116879804486

In [1]:
icc(ie2, :dept)

0.016119387658148597

In [1]:
vif(ie1)

9-element Vector{Float64}:
 1.514189953373784
 1.7354052063945073
 1.782230800039364
 1.4493788224538782
 1.43808916649895
 1.594896527717326
 1.4634020652231683
 1.8267101123046552
 1.0161785707996789

In [1]:
DataFrame(; coef=fixefnames(ie1)[2:end], VIF=vif(ie1))

In [1]:
gvif(ie1)

3-element Vector{Float64}:
 1.3110868052852684
 1.3257307677249577
 1.016178570799679

In [1]:
DataFrame(; term=termnames(ie1)[2][2:end], GVIF=gvif(ie1))

# RegressionFormulae.jl

<https://github.com/kleinschmidt/RegressionFormulae.jl>

RegressionFormulae.jl provides a few extensions to the somewhat more
restricted variant of the Wilkinson-Roger notation found in Julia. In
particular, it adds `/` for nested designs within the fixed effects and
`^` for computing interactions only up to a certain order.

In [1]:
using RegressionFormulae

fit(MixedModel,
          @formula(y ~ 1 + service / (studage + lectage) +
                      (1 | s) +
                      (1 | d) +
                      (1 | dept)),
          insteval; progress)

In [1]:
fit(MixedModel,
          @formula(y ~ 1 + (studage + lectage + service)^2 +
                      (1 | s) +
                      (1 | d) +
                      (1 | dept)),
          insteval; progress)

# BoxCox.jl

<https://palday.github.io/BoxCox.jl/v0.3/>

BoxCox.jl implements a the Box-Cox transformation in an efficient way.
Via package extensions, it supports specializations for MixedModels.jl
and several plotting functions, but does not incur a dependency penalty
for this functionality when MixedModels.jl or Makie.jl are not loaded.

In [1]:
using BoxCox

bc = fit(BoxCoxTransformation, ss2)

Minimizing 2    Time: 0:00:00 (65.20 ms/it)
  objective:  -277.07360476082556
Minimizing 57    Time: 0:00:00 (17.21 ms/it)

Box-Cox transformation

estimated λ: -1.0747
resultant transformation:

 y^-1.1 - 1
------------
    -1.1

In [1]:
using CairoMakie
boxcoxplot(bc; conf_level=0.95)

The estimated λ is very close to -1, i.e. the reciprocal of reaction
time, which has a natural interpretation as speed. In other words, the
Box-Cox transformation suggests that we should consider modelling the
sleepstudy data as speed (reaction per unit time) instead of reaction
time:

In [1]:
fit(MixedModel, @formula(1000 / reaction ~ 1 + days + (1 + days|subj)), sleepstudy)

(We multiply by 1000 to get the responses per *second* instead of the
responses per *millisecond*.)

> **Tip**
>
> BoxCox.jl also works with classical linear models.

# Effects.jl

<https://beacon-biosignals.github.io/Effects.jl/v1.2/>

Effects.jl provides a convenient method to compute *effects*,
i.e. predictions and associated prediction intervals computed at points
on a reference grid. For models with a nonlinear link function,
Effects.jl will also compute appropriate errors on the response scale
based on the difference method.

For MixedModels.jl, the predictions are computed based on the fixed
effects only.

The functionality of Effects.jl was inspired by the `effects` and
`emmeans` packages in R and the methods within are based on Fox (2003).

In [1]:
using Effects

In [1]:
design = Dict(:age => -15:1:20,
              :anych => [true, false])

eff_logit = effects(design, gm1; eff_col="use", level=0.95)

In [1]:
eff_prob = effects(design, gm1; eff_col="use", level=0.95, invlink=AutoInvLink())

Effects are particularly nice for visualizing the model fit and its
predictions.

In [1]:
using AlgebraOfGraphics # like ggplot2, but an algebra instead of a grammar
using CairoMakie

plt1 = data(eff_logit) *
      mapping(:age, :use; color=:anych) *
      (visual(Lines) + mapping(; lower=:lower, upper=:upper) * visual(LinesFill))
draw(plt1)

In [1]:
plt2 = data(eff_prob) *
      mapping(:age, :use; color=:anych => "children") *
      (visual(Lines) + mapping(; lower=:lower, upper=:upper) * visual(LinesFill))
draw(plt2)

In [1]:
using Statistics: mean
contra_by_age = transform(contra,
                          :age => ByRow(x -> round(Int, x)),
                          :use => ByRow(==("Y"));
                          renamecols=false)
contra_by_age = combine(groupby(contra_by_age, [:age, :anych]),
                        :use => mean => :use)
plt3 = plt2 +
       data(contra_by_age) *
       mapping(:age, :use;
               color=:anych => "children") * visual(Scatter)

draw(plt3;
     axis=(; title="Estimated contraceptive use by age and children",
            limits=(nothing, (0, 1)) # ylim=0,1, xlim=auto
            ))

Effects and estimated marginal (least squares) means are closely related
and partially concepts. Effects.jl provides convenience function
`emmeans` and `empairs` for computing EM means and pairwise differences
of EM means.

In [1]:
emmeans(gm1)

In [1]:
empairs(gm1; dof=Inf)

> **Tip**
>
> Effects.jl will work with any package that supports the
> StatsAPI.jl-based `RegressionModel` interface.

# StandardizedPredictors.jl

<https://beacon-biosignals.github.io/StandardizedPredictors.jl/v1/>

StandardizedPredictors.jl provides a convenient way to express
centering, scaling, and z-standardization as a “contrast” via the
pseudo-contrasts `Center`, `Scale`, `ZScore`. Because these use the
usual contrast machinery, they work well with any packages that use that
machinery correctly (e.g. Effects.jl). The default behavior is to
empirically compute the center and scale, but these can also be
explicitly provided, either as a number or as a function (e.g. `median`
to use the median for centering.)

In [1]:
using StandardizedPredictors

contrasts = Dict(:days => Center())
fit(MixedModel,
    @formula(reaction ~ 1 + days + (1 + days|subj)), sleepstudy;
    contrasts)

> **Tip**
>
> StandardizedPredictors.jl will work with any package that supports the
> StatsModels.jl-based `@formula` and contrast machinery.

# RCall.jl and JellyMe4.jl

<https://juliainterop.github.io/RCall.jl/stable/>

<https://github.com/palday/JellyMe4.jl/>

RCall.jl provides a convenient interface for interoperability with R
from Julia. JellyMe4.jl extends the functionality of RCall so that
MixedModels.jl-fitted models and lme4-fitted models can be translated to
each other. In practical terms, this means that you can enjoy the speed
of Julia for model fitting, but use all the extra packages you love from
R’s larger ecosystem.

# References

Fox, J. (2003). Effect Displays in *r* for Generalised Linear Models.
*Journal of Statistical Software*, *8*(15).
<https://doi.org/10.18637/jss.v008.i15>